# Import Packages and Files

In [1]:
import dataloader as dl

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 5000, 128)

ValueError: sampler option is mutually exclusive with shuffle